<a href="https://colab.research.google.com/github/alexapruiz/Projetos/blob/master/Python/Colab/Titanic2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importa as bibliotecas
import pandas as pd
import numpy as np

In [ ]:
#Cria a função que vai gerar uma coluna binária para o sexo (Male -> 0 e Female -> 1)
def transformar_sexo(valor):
  if valor == 'female':
    return 1
  else:
    return 0

In [ ]:
#Importa os arquivos do Kaggle
train = pd.read_csv("train.csv", sep=';')
teste = pd.read_csv("test.csv", sep=';')

In [ ]:
train

In [ ]:
#Criando o campo 'Sexo_binario'
train['Sex_binario'] = train['Sex'].map(transformar_sexo)
teste['Sex_binario'] = teste['Sex'].map(transformar_sexo)

#Criando outros campos para tentar melhorar o modelo
train['Embarked_S'] = (train['Embarked'] == 'S').astype(int)
train['Embarked_C'] = (train['Embarked'] == 'C').astype(int)
train['Embarked_Q'] = (train['Embarked'] == 'Q').astype(int)
train['Cabine_Nula'] = train['Cabin'].isnull().astype(int)
train['Nome_Contem_Miss'] = train['Name'].str.contains('Miss').astype('int')
train['Nome_Contem_Mrs'] = train['Name'].str.contains('Mrs').astype('int')
train['Nome_Contem_Master'] = train['Name'].str.contains('Master').astype('int')
train['Nome_Contem_Col'] = train['Name'].str.contains('Col').astype('int')
train['Nome_Contem_Major'] = train['Name'].str.contains('Major').astype('int')
train['Nome_Contem_Mr'] = train['Name'].str.contains('Mr').astype('int')

#Definindo os campos a serem analisados
#variaveis = ['Sex_binario', 'Age', 'Pclass', 'SibSp','Parch', 'Fare']
variaveis = ['PassengerId','Sex_binario', 'Age', 'Pclass', 'SibSp','Parch', 'Fare','Embarked_S','Embarked_C','Embarked_Q','Cabine_Nula','Nome_Contem_Miss','Nome_Contem_Mrs','Nome_Contem_Master','Nome_Contem_Col','Nome_Contem_Major','Nome_Contem_Mr']

#Substituindo dados Nan por -1
X = train[variaveis].fillna(-1)
y = train['Survived']

In [ ]:
#Importando as bibliotecas de Machine Learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold

In [ ]:
#Usando uma técnica de reamostragem -> Validação Cruzada
resultados = []
kf = RepeatedKFold(n_splits=3, n_repeats=40, random_state=1)

for linhas_treino, linhas_valid in kf.split(X):
  #print('Treino: ', linhas_treino.shape[0])
  #print('Teste: ', linhas_valid.shape[0])

  X_treino , X_valid = X.iloc[linhas_treino], X.iloc[linhas_valid]
  y_treino , y_valid = y.iloc[linhas_treino], y.iloc[linhas_valid]

  modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
  modelo.fit(X_treino, y_treino)
  p = modelo.predict(X_valid)
  acc = np.mean(y_valid == p)
  resultados.append(acc)
  print('Acc: ',acc)  

In [ ]:
resultados

In [ ]:
#Resultados : - n_splits=3, n_repeats=20, random_state=20 -> 0.8108866442199776
#Resultados : - n_splits=4, n_repeats=40, random_state=10 -> 0.8096018664404315
#Resultados : - n_splits=4, n_repeats=40, random_state=1  -> 0.8119326445279361

np.mean(resultados)

0.819304152637486

In [ ]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
pylab.hist(resultados)

# Aplicando o modelo para gerar novo resultado

In [ ]:
modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
modelo.fit(X, y)

In [ ]:
teste['Embarked_S'] = (teste['Embarked'] == 'S').astype(int)
teste['Embarked_C'] = (teste['Embarked'] == 'C').astype(int)
teste['Embarked_Q'] = (teste['Embarked'] == 'Q').astype(int)
teste['Cabine_Nula'] = teste['Cabin'].isnull().astype(int)
teste['Nome_Contem_Miss'] = teste['Name'].str.contains('Miss').astype('int')
teste['Nome_Contem_Mrs'] = teste['Name'].str.contains('Mrs').astype('int')
teste['Nome_Contem_Master'] = teste['Name'].str.contains('Master').astype('int')
teste['Nome_Contem_Col'] = teste['Name'].str.contains('Col').astype('int')
teste['Nome_Contem_Major'] = teste['Name'].str.contains('Major').astype('int')
teste['Nome_Contem_Mr'] = teste['Name'].str.contains('Mr').astype('int')

variaveis = ['PassengerId','Sex_binario', 'Age', 'Pclass', 'SibSp','Parch', 'Fare','Embarked_S','Embarked_C','Embarked_Q','Cabine_Nula','Nome_Contem_Miss','Nome_Contem_Mrs','Nome_Contem_Master','Nome_Contem_Col','Nome_Contem_Major','Nome_Contem_Mr']

In [ ]:
valida = teste[variaveis]
valida = valida.fillna(-1)
p2 = modelo.predict(valida)

In [ ]:
# Gerando as previsões
sub = pd.Series(p2, index=teste['PassengerId'], name='Survived')
sub

In [ ]:
# Criando um arquivo csv com o resultado
sub.to_csv('Modelo_com_todos_atributos.csv', header=True)